In [72]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#filename = r"C:\Exoplanet\PS_2026.01.15_09.23.48.csv"
filename = "C:\Exoplanet\composite_exo_data-unfiltered.csv"
df = pd.read_csv(filename, comment='#', low_memory=False) # comment="#" skips NASA metadata lines
# low_memory=False avoids mixed dtype warnings
#print(df.columns)

df_filtered = df[
    (
        df["pl_rade"].between(1.5, 3) &
        df["pl_bmasse"].notna() &
        df["st_teff"].notna() &
        df["st_rad"].notna() &
        df["pl_orbsmax"].notna() &
        df["sy_jmag"].notna() &
        df["pl_trandur"].notna() &
        df["pl_bmasseerr1"].notna() &
        df["pl_bmasseerr2"].notna() 
        #df["default_flag"].between(1,1)  
        
        
         
       
        
         
    )
].copy()  # Filter the table to select sub-Neptunes and removes rows with missing values
# .copy() avoids pandas "SettingWithCopyWarning"

Rp = df_filtered["pl_rade"].values  # planet radius
Mp = df_filtered["pl_bmasse"].values # planet mass
starsN= df_filtered["sy_snum"]  #number of stars
Tstar = df_filtered["st_teff"].values       # Eq temp of star in K
Rstar = df_filtered["st_rad"].values        # temp of star in R_sun
a = df_filtered["pl_orbsmax"].values        # Orbital semimajor axis in AU
Jmag = df_filtered["sy_jmag"].values
transit = df_filtered["pl_trandur"].values
upperunc=df_filtered["pl_bmasseerr1"].values
lowerunc=df_filtered["pl_bmasseerr2"].values 
Rstar_AU = Rstar * 0.00465047  # Conversion of radius of star into AU

Teq = Tstar * np.sqrt(Rstar_AU / (2 * a))  #Equilibrium temperature

df_filtered["Teq_calc"] = Teq
Teq = df_filtered["Teq_calc"].values

def scale_factor(Rp): #code which gives scale factor values depending on radius
    if Rp < 1.5:
        return 0.19
    elif Rp < 2.75:
        return 1.26
    elif Rp < 4.0:
        return 1.28
    else:
        return 1.15

S = np.array([scale_factor(r) for r in Rp])  #scale factors for each planet

TSM = S * (Rp**3 * Teq) / (Mp * Rstar**2) * 10**(-Jmag/5) # transmission spectroscopy metric formula

df_filtered["TSM"] = TSM  #stores TSM values in the filtered table

df_sorted = df_filtered.sort_values("TSM", ascending=False)  # ranks planets by TSM values with highest (best) first

ranked = df_sorted[["pl_name", "pl_rade", "pl_bmasse", "Teq_calc", "TSM" , "pl_trandur"]]

print(ranked.head(20)) #leftmost column shows position in the dataframe for each row

print("Number of filtered planets is",len(df_filtered))

top25 = df_sorted.head(25)
total_duration_hours = top25["pl_trandur"].sum()
print("Total transit duration of top 25 TSM planets:", total_duration_hours, "hours")

bottom25 = df_sorted.tail(25)
total_duration_hours_bottom = bottom25["pl_trandur"].sum()
print("Total transit duration of bottom 25 TSM planets:", total_duration_hours_bottom, "hours")

observationtime=(total_duration_hours + total_duration_hours_bottom )*2
print("Total observing time if each transit is measured once is",observationtime)

          pl_name   pl_rade  pl_bmasse     Teq_calc         TSM  pl_trandur
213     GJ 1214 b  2.733000       8.41   566.754252  416.080011    0.869660
4856    L 98-59 d  1.627000       1.64   416.160006  358.385945    0.840000
958   HD 219134 b  1.602000       4.74  1015.165348  293.060529    0.945000
6062     pi Men c  2.018900       3.63  1196.290120  265.138709    2.952024
36       55 Cnc e  1.875000       7.99  1949.055205  253.529405    1.543900
664   HD 136352 c  2.916000      11.24   677.187712  234.932960    3.251000
861   HD 191939 d  2.995000       2.80   541.495873  227.605033    5.360000
1402     HR 858 b  2.004000       3.55  1554.373409  225.646925    2.705000
242     GJ 3090 b  2.130000       3.34   692.361917  220.392422    1.281000
959   HD 219134 c  1.511000       4.36   782.117635  205.963533    1.660000
1020  HD 260655 c  1.533000       3.09   557.559123  196.607652    0.980000
5644    TOI-544 b  2.018000       2.89  1057.835373  186.757555    1.210000
5145   TOI-1